# Bluesky Ranker — Example Notebook

This notebook demonstrates the typical workflow:
- Fetch recent public posts into SQLite (upsert-by-URI)
- Load posts from SQLite into a Polars DataFrame
- Rank posts using the TopicRanker (TF–IDF/Count/SBERT)
- Inspect the top clusters and sample posts
- (Optional) Generate a per-handle cluster report to Markdown


> Note: This notebook expects a SQLite DB with posts.
Create one via the sample (no network): `python -m blueskyranker.sample_db --db newsflows_sample.db`
or fetch live data via the fetcher CLI.


In [ ]:
# Imports and setup
import polars as pl
from blueskyranker.fetcher import Fetcher, ensure_db, load_posts_df
from blueskyranker.ranker import TopicRanker


## 1) Fetch recent posts into SQLite

- Adjust `--max-age-days` to control the time window.
- Upsert ensures engagement metrics refresh over time.
- You can also call the fetcher via CLI if you prefer.


In [ ]:
fetcher = Fetcher()
result = fetcher.fetch(max_age_days=7)  # change to your needs
print(result)


## 2) Load posts from SQLite

- Choose a handle you want to rank.
- You can limit rows or change ordering as needed.


In [ ]:
conn = ensure_db('newsflows.db')
handle = 'news-flows-nl.bsky.social'  # pick one of your handles
data = load_posts_df(conn, handle=handle, limit=2000, order_by='createdAt', descending=False)
data.head()


## 3) Rank posts by topic

- Methods: `networkclustering-tfidf`, `networkclustering-count`, `networkclustering-sbert` (slower, higher semantic quality).
- `similarity_threshold`: raise for fewer/tighter clusters.
- `vectorizer_stopwords`: 'english' | list of words | None.


In [ ]:
ranker = TopicRanker(
    returnformat='dataframe',
    method='networkclustering-tfidf',  # try 'networkclustering-sbert' for semantics
    descending=True,
    similarity_threshold=0.2,
    vectorizer_stopwords='english',
    # Optional windows (days):
    cluster_window_days=7,
    engagement_window_days=3,
    push_window_days=1,
)
ranking = ranker.rank(data)
ranking.head()


## 4) Inspect top clusters and posts

- We show the 3 most engaged clusters.
- For each, we list the 5 most recent posts with key fields.


In [ ]:
clusters = (
    ranking.group_by('cluster')
    .agg([
        pl.col('cluster_size').first().alias('size'),
        pl.col('cluster_engagement_count').first().alias('engagement')
    ])
    .sort('engagement', descending=True)
    .head(3)
)
for row in clusters.iter_rows(named=True):
    cid = row['cluster']
    size = int(row['size']) if row['size'] is not None else 0
    eng = int(row['engagement']) if row['engagement'] is not None else 0
    print(f"\n=== Cluster {cid} | size={size} | engagement={eng}")
    subset = (
        ranking.filter(pl.col('cluster') == cid)
        .sort('createdAt', descending=True)
        .head(5)
    )
    for rec in subset.select(['uri','text','news_title','news_description','news_uri']).iter_rows(named=True):
        print(f"- uri: {rec['uri']}")
        print(f"  text: {rec.get('text')}")
        print(f"  news_title: {rec.get('news_title')}")
        print(f"  news_description: {rec.get('news_description')}")
        print(f"  news_uri: {rec.get('news_uri')}")


## 5) (Optional) Generate a cluster report

- This writes `cluster_report.md` with top clusters per handle.
- You can adjust method, threshold, and stopwords.


## 6) (Optional) End-to-end: fetch → rank → push (per handle)

- Runs the whole flow and logs a short cluster summary to `push.log`.


In [ ]:
from blueskyranker.pipeline import run_fetch_rank_push
run_fetch_rank_push(
    handles=[handle],
    method='networkclustering-tfidf', similarity_threshold=0.2,
    cluster_window_days=7, engagement_window_days=3, push_window_days=1,
    include_pins=False, test=True, log_path='push.log'
)


In [ ]:
from blueskyranker.cluster_report import generate_cluster_report
generate_cluster_report(db_path='newsflows.db', output_path='cluster_report.md',
                        method='networkclustering-sbert', sample_max=300,
                        similarity_threshold=0.2, vectorizer_stopwords='english')
print('Wrote cluster_report.md')
